In [1]:
import pandas as pd
import numpy as np
import random
import gc
import seaborn as sns
import lightgbm as lgb
from matplotlib import pyplot as plt
# import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.metrics import recall_score, auc, accuracy_score, f1_score, precision_score, classification_report, roc_auc_score,mean_squared_error
from pandas.api.types import is_categorical_dtype
from pandas.api.types import is_datetime64_any_dtype as is_datetime
# import missingno as msno
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

In [107]:
data = pd.read_pickle('./data_2d.pickle')
select_features = ['user_id','cp_server_no','cp_role_id','role_created_login_num',
       'role_created_active', 'role_created_online',
       'max_role_level', 'ip_num',
       'device_num',
       'pay_num', 'pay_sum', 'active_0-8', 'active_8-12', 'active_12-14',
       'active_14-18', 'active_18-24', 'pay_grade_1', 'pay_grade_2',
       'pay_grade_3', 'pay_grade_4', 'pay_grade_5', 'pay_grade_6',
       'pay_rate', 'pay_avg',
       'model_money_level', 'hour', 'weekend', 'is_holidays','mobile','platform',
       'user_creates_3_server_num','user_creates_3_role_num', 'time_interval']
label = ['role_created_30_pay_sum']
df = data[select_features].join(data[label])
df['role_created_30_pay_sum'].fillna(0,inplace=True)
df['platform'].fillna(0,inplace=True)
df['user_creates_3_server_num'].fillna(1,inplace=True)
df['user_creates_3_role_num'].fillna(1,inplace=True)
df['time_interval'].fillna(0,inplace=True)
df['model_money_level'].fillna(4,inplace=True)

In [108]:
df_pay = df[df['pay_sum']>0]
df_nopay = df[df['pay_sum']==0]
df_nopay_online_n = df_nopay[(df_nopay['role_created_online']<400) | (df_nopay['role_created_login_num']<3)|(df_nopay['max_role_level']<2)]
df_nopay_online_y = df_nopay.loc[np.setdiff1d(df_nopay.index, df_nopay_online_n.index),:]

In [109]:
df_pay['role_created_30_pay_sum'].sum()

90415319.0

In [110]:
df_nopay['role_created_30_pay_sum'].sum()

9505942.0

In [111]:
df_pay['role_created_30_pay_sum'].sum()/(df_pay['role_created_30_pay_sum'].sum()+df_nopay['role_created_30_pay_sum'].sum())

0.9048656721816191

In [112]:
print('前n天没有付费的人数：',df_nopay.shape[0])
print('前n天没有付费且不活跃的用户:',df_nopay_online_n.shape[0])
print('前n天没有付费但活跃的用户:',df_nopay_online_y.shape[0])

前n天没有付费的人数： 5051238
前n天没有付费且不活跃的用户: 3684538
前n天没有付费但活跃的用户: 1366700


In [113]:
print('前n天没有付费且不活跃的用户30天付费金额:', df_nopay_online_n['role_created_30_pay_sum'].sum())
print('前n天没有付费但活跃的用户30天付费金额:', df_nopay_online_y['role_created_30_pay_sum'].sum())
print('前n天没有付费但前30天付费的用户人数:',df_nopay_online_y[df_nopay_online_y['role_created_30_pay_sum']>0].shape[0])
print('前n天没有付费但前30天付费的用户人数前30天付费金额:',df_nopay_online_y[df_nopay_online_y['role_created_30_pay_sum']>0]['role_created_30_pay_sum'].mean())

前n天没有付费且不活跃的用户30天付费金额: 848848.0
前n天没有付费但活跃的用户30天付费金额: 8657094.0
前n天没有付费但前30天付费的用户人数: 34703
前n天没有付费但前30天付费的用户人数前30天付费金额: 249.462409589949


In [168]:
df_train, df_test = train_test_split(df,test_size=0.3)
df_train_pay = df_train[df_train['pay_sum']>0]
df_train_nopay = df_train[df_train['pay_sum']==0]
df_train_nopay_online_n = df_train_nopay[(df_train_nopay['role_created_online']<400) | (df_train_nopay['role_created_login_num']<3)|(df_train_nopay['max_role_level']<2)]
df_train_nopay_online_y = df_train_nopay.loc[np.setdiff1d(df_train_nopay.index, df_train_nopay_online_n.index),:]

In [174]:
del df_nopay,df_pay,df_nopay_online_n,df_nopay_online_y
gc.collect()

NameError: name 'df_nopay' is not defined

## 在前n天没有付费但活跃的用户中，预测前30天是否会成为付费用户

In [175]:
df_train_nopay_online_y.shape

(957074, 34)

In [176]:
df_train_nopay_online_y['pay_30'] = (df_train_nopay_online_y['role_created_30_pay_sum']>0).astype(int)
df_train_nopay_online_y.drop(['role_created_30_pay_sum'], axis=1, inplace=True)
df_train_nopay_online_y.head()

,user_id,cp_server_no,cp_role_id,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,device_num,pay_num,...,model_money_level,hour,weekend,is_holidays,mobile,platform,user_creates_3_server_num,user_creates_3_role_num,time_interval,pay_30
4,90001842,660011,12927599,8.28,1.0,2437.000000,18.0,1.0,1.0,0.0,...,3.0,13,4,0,0,1.0,1.0,1.0,0.0,0
7,90001891,660011,12927602,5.52,1.0,4363.865067,12.0,1.0,1.0,0.0,...,4.0,13,4,0,0,1.0,1.0,1.0,0.0,0
18,90002408,660011,12927613,7.36,1.0,5818.486756,16.0,1.0,1.0,0.0,...,4.0,13,4,0,0,1.0,1.0,1.0,0.0,0
19,90002420,660011,12927614,3.00,2.0,4286.000000,21.0,2.0,2.0,0.0,...,4.0,13,4,0,0,1.0,1.0,1.0,0.0,0
24,90002604,660011,12927619,5.52,1.0,4363.865067,12.0,1.0,1.0,0.0,...,2.0,13,4,0,0,1.0,1.0,1.0,0.0,0


In [177]:
df_train_nopay_online_y['pay_30'].value_counts()

0    932745
1     24329
Name: pay_30, dtype: int64

In [178]:
target = df_train_nopay_online_y['pay_30']
features = df_train_nopay_online_y.drop(['pay_30','user_id','cp_server_no','cp_role_id'], axis=1)

from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold()         #实例化，不填参数默认方差为0
features = selector.fit_transform(features)         #获取删除不合格特征之后的新特征矩阵

X_val, x_test, Y_val, y_test = train_test_split(features,target, test_size=0.3)

In [179]:
from imblearn.over_sampling import SMOTE
smo = SMOTE(random_state=0)
X_val, Y_val = smo.fit_sample(X_val, Y_val)

In [180]:
# 不调参
params = {
"objective": "binary",
"boosting_type": "gbdt",
"learning_rate": 0.03,
"max_depth":3,
"num_leaves":35,
"min_child_samples":15,
"feature_fraction": 0.7,
"bagging_fraction":0.7,
"bagging_freq":10,
"lambda_l1":2,
"lambda_l2":2,
"metric": "auc",
# "is_unbalance":True
}

train_data = lgb.Dataset(X_val, label=Y_val)
val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
lgb_b = lgb.train(params, train_data, num_boost_round=8000, early_stopping_rounds=100, valid_sets=[train_data, val_data])

[1]	training's auc: 0.899543	valid_1's auc: 0.876517
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.92633	valid_1's auc: 0.884397
[3]	training's auc: 0.929681	valid_1's auc: 0.884481
[4]	training's auc: 0.931919	valid_1's auc: 0.88247
[5]	training's auc: 0.936568	valid_1's auc: 0.880329
[6]	training's auc: 0.938192	valid_1's auc: 0.885671
[7]	training's auc: 0.940656	valid_1's auc: 0.885824
[8]	training's auc: 0.941007	valid_1's auc: 0.884913
[9]	training's auc: 0.941785	valid_1's auc: 0.884872
[10]	training's auc: 0.942726	valid_1's auc: 0.886819
[11]	training's auc: 0.942844	valid_1's auc: 0.886422
[12]	training's auc: 0.944122	valid_1's auc: 0.885857
[13]	training's auc: 0.944322	valid_1's auc: 0.885604
[14]	training's auc: 0.944571	valid_1's auc: 0.886006
[15]	training's auc: 0.944679	valid_1's auc: 0.885096
[16]	training's auc: 0.946501	valid_1's auc: 0.885998
[17]	training's auc: 0.946524	valid_1's auc: 0.886055
[18]	training's auc: 0.947665	v

[152]	training's auc: 0.984705	valid_1's auc: 0.886476
[153]	training's auc: 0.984801	valid_1's auc: 0.886393
[154]	training's auc: 0.985074	valid_1's auc: 0.886368
[155]	training's auc: 0.9851	valid_1's auc: 0.886466
[156]	training's auc: 0.985193	valid_1's auc: 0.886357
[157]	training's auc: 0.98542	valid_1's auc: 0.886317
[158]	training's auc: 0.985558	valid_1's auc: 0.886252
[159]	training's auc: 0.985844	valid_1's auc: 0.886322
[160]	training's auc: 0.986051	valid_1's auc: 0.886282
[161]	training's auc: 0.986127	valid_1's auc: 0.886234
[162]	training's auc: 0.986233	valid_1's auc: 0.886208
[163]	training's auc: 0.98634	valid_1's auc: 0.886255
[164]	training's auc: 0.986524	valid_1's auc: 0.886185
[165]	training's auc: 0.986759	valid_1's auc: 0.886264
[166]	training's auc: 0.986872	valid_1's auc: 0.886197
[167]	training's auc: 0.987047	valid_1's auc: 0.886139
[168]	training's auc: 0.987084	valid_1's auc: 0.886112
[169]	training's auc: 0.987143	valid_1's auc: 0.886251
[170]	training

# 前2天有付费的用户进行回归

In [123]:
df_train_pay.head()

,user_id,cp_server_no,cp_role_id,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,device_num,pay_num,...,model_money_level,hour,weekend,is_holidays,mobile,platform,user_creates_3_server_num,user_creates_3_role_num,time_interval,role_created_30_pay_sum
2942262,108232096,660534,16219058,5.0,2.0,20320.0,41.00000,1.0,1.0,3.0,...,4.0,22,0,0,0,1.0,1.0,1.0,0.0,110.0
1514651,104233510,S335,14832548,40.0,2.0,54493.0,57.00000,4.0,1.0,6.0,...,4.0,17,6,0,0,1.0,1.0,1.0,0.0,512.0
3698207,54935389,660671,16938495,10.0,2.0,57600.0,57.00000,4.0,1.0,7.0,...,6.0,16,0,0,0,0.0,1.0,1.0,0.0,474.0
4714037,112489997,660855,17482305,26.0,2.0,44131.0,55.00000,10.0,1.0,1.0,...,1.0,19,1,0,0,1.0,1.0,1.0,6.0,170.0
2197270,106190435,660428,15497947,40.0,2.0,11515.0,29.12532,5.0,1.0,1.0,...,10.0,18,6,0,0,2.0,1.0,1.0,0.0,268.0


In [182]:
target = df_train_pay['role_created_30_pay_sum']
features = df_train_pay.drop(['role_created_30_pay_sum','user_id','cp_server_no','cp_role_id'], axis=1)
target_ln = np.log1p(target)
X_val, x_test, Y_val, y_test = train_test_split(features,target_ln, test_size=0.3)

In [125]:
# lgb的调参与交叉验证
train_data = lgb.Dataset(X_val, label=Y_val)
val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)

# 设置参数初始值，不含交叉验证参数
print('设置参数')
params = {
    'boosting_type':'gbdt',
    'objective':'regression',
    'metric':'rmse',
    'nthread':4,
    'learning_rate':0.05
}

# 交叉验证（调参）
print('交叉验证')
min_rmse = float('5')
best_params = {}

# 准确率
print('调参1：提高准确率')
for num_leaves in range(5, 100, 5):
    for max_depth in range(3, 8, 1):
        params['num_leaves'] = num_leaves
        params['max_depth'] = max_depth
        
        cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50     
                            )
        mean_rmse = pd.Series(cv_results['rmse-mean']).min()
        boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
        if mean_rmse <= min_rmse:
            min_rmse = mean_rmse
            best_params['num_leaves'] = num_leaves
            best_params['max_depth'] = max_depth

if 'num_leaves' and 'max_depth' in best_params.keys():
    params['num_leaves'] = best_params['num_leaves']
    params['max_depth'] = best_params['max_depth']

# 过拟合
print('调参2：降低过拟合')
for max_bin in range(5, 256, 10):
    for min_data_in_leaf in range(1, 102, 10):
        params['max_bin'] = max_bin
        params['min_data_in_leaf'] = min_data_in_leaf
        
        cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50 
                            )
        mean_rmse = pd.Series(cv_results['rmse-mean']).min()
        boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
        if mean_rmse <= min_rmse:
            min_rmse = mean_rmse
            best_params['min_data_in_leaf'] = min_data_in_leaf
            best_params['max_bin'] = max_bin
            
if 'max_bin' and 'min_data_in_leaf' in best_params.keys():
    params['max_bin'] = best_params['max_bin']
    params['min_data_in_leaf'] = best_params['min_data_in_leaf']

print('调参3：降低过拟合')
for feature_fraction in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    for bagging_fraction in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for bagging_freq in range(0,50,5):
            params['feature_fraction'] = feature_fraction
            params['bagging_fraction'] = bagging_fraction
            params['bagging_freq'] = bagging_freq

            cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50
                            )
            mean_rmse = pd.Series(cv_results['rmse-mean']).min()
            boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
            if mean_rmse <= min_rmse:
                min_rmse = mean_rmse
                best_params['feature_fraction'] = feature_fraction
                best_params['bagging_fraction'] = bagging_fraction
                best_params['bagging_freq'] = bagging_freq

if 'feature_fraction' and 'bagging_fraction' and 'bagging_freq' in best_params.keys():
    params['feature_fraction'] = best_params['feature_fraction']
    params['bagging_fraction'] = best_params['bagging_fraction']
    params['bagging_freq'] = best_params['bagging_freq']

print('调参4：降低过拟合')

for lambda_l1 in [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0]:
    for lambda_l2 in [1e-5,1e-3,1e-1,0.0,0.1,0.4,0.6,0.7,0.9,1.0]:
        params['lambda_l1'] = lambda_l1
        params['lambda_l2'] = lambda_l2
        cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50
                            )
        mean_rmse = pd.Series(cv_results['rmse-mean']).min()
        boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
        if mean_rmse <= min_rmse:
            min_rmse = mean_rmse
            best_params['lambda_l1'] = lambda_l1
            best_params['lambda_l2'] = lambda_l2
if 'lambda_l1' and 'lambda_l2' in best_params.keys():
    params['lambda_l1'] = best_params['lambda_l1']
    params['lambda_l2'] = best_params['lambda_l2']

print("调参5：降低过拟合2")
for min_split_gain in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    params['min_split_gain'] = min_split_gain
    
    cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50    
                            )
    mean_rmse = pd.Series(cv_results['rmse-mean']).min()
    boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
    if mean_rmse >= min_rmse:
        min_rmse = mean_rmse
        
    best_params['min_split_gain'] = min_split_gain
if 'min_split_gain' in best_params.keys():
    params['min_split_gain'] = best_params['min_split_gain']
print(best_params)

设置参数
交叉验证
调参1：提高准确率
[50]	cv_agg's rmse: 1.02054 + 0.00767231
[100]	cv_agg's rmse: 0.995009 + 0.00904712
[50]	cv_agg's rmse: 1.02054 + 0.00767231
[100]	cv_agg's rmse: 0.994833 + 0.00896952
[50]	cv_agg's rmse: 1.02054 + 0.00767231
[100]	cv_agg's rmse: 0.994833 + 0.00896952
[50]	cv_agg's rmse: 1.02054 + 0.00767231
[100]	cv_agg's rmse: 0.994833 + 0.00896952
[50]	cv_agg's rmse: 1.02054 + 0.00767231
[100]	cv_agg's rmse: 0.994833 + 0.00896952
[50]	cv_agg's rmse: 1.01251 + 0.0075445
[100]	cv_agg's rmse: 0.993245 + 0.00880601
[50]	cv_agg's rmse: 1.00857 + 0.00797949
[100]	cv_agg's rmse: 0.991903 + 0.00909924
[50]	cv_agg's rmse: 1.00839 + 0.00787953
[100]	cv_agg's rmse: 0.991948 + 0.00906225
[50]	cv_agg's rmse: 1.00843 + 0.00793518
[100]	cv_agg's rmse: 0.991856 + 0.00906595
[50]	cv_agg's rmse: 1.00843 + 0.00793518
[100]	cv_agg's rmse: 0.991779 + 0.0089407
[50]	cv_agg's rmse: 1.01251 + 0.0075445
[100]	cv_agg's rmse: 0.993245 + 0.00880601
[50]	cv_agg's rmse: 1.00512 + 0.00805605
[100]	cv_agg's rms

[50]	cv_agg's rmse: 1.0015 + 0.00837332
[100]	cv_agg's rmse: 0.991013 + 0.00923856
[50]	cv_agg's rmse: 1.00144 + 0.00845832
[100]	cv_agg's rmse: 0.990716 + 0.00928783
[50]	cv_agg's rmse: 1.00149 + 0.00841725
[100]	cv_agg's rmse: 0.990835 + 0.00907928
[50]	cv_agg's rmse: 1.00161 + 0.00839547
[100]	cv_agg's rmse: 0.990934 + 0.00908839
[50]	cv_agg's rmse: 1.00161 + 0.00840504
[100]	cv_agg's rmse: 0.990962 + 0.00909816
[50]	cv_agg's rmse: 1.00155 + 0.00842064
[100]	cv_agg's rmse: 0.990961 + 0.00908677
[50]	cv_agg's rmse: 1.0014 + 0.00851521
[100]	cv_agg's rmse: 0.990836 + 0.0093396
[50]	cv_agg's rmse: 1.00173 + 0.00832245
[100]	cv_agg's rmse: 0.991121 + 0.00927051
[50]	cv_agg's rmse: 1.00173 + 0.0086257
[100]	cv_agg's rmse: 0.991155 + 0.00973057
[50]	cv_agg's rmse: 1.00155 + 0.0084091
[100]	cv_agg's rmse: 0.99055 + 0.00908532
[50]	cv_agg's rmse: 1.00149 + 0.00837795
[100]	cv_agg's rmse: 0.990758 + 0.00923502
[50]	cv_agg's rmse: 1.0015 + 0.00837332
[100]	cv_agg's rmse: 0.991013 + 0.00923856

[100]	cv_agg's rmse: 0.990758 + 0.00923502
[50]	cv_agg's rmse: 1.0015 + 0.00837332
[100]	cv_agg's rmse: 0.991013 + 0.00923856
[50]	cv_agg's rmse: 1.00144 + 0.00845832
[100]	cv_agg's rmse: 0.990716 + 0.00928783
[50]	cv_agg's rmse: 1.00149 + 0.00841725
[100]	cv_agg's rmse: 0.990835 + 0.00907928
[50]	cv_agg's rmse: 1.00161 + 0.00839547
[100]	cv_agg's rmse: 0.990934 + 0.00908839
[50]	cv_agg's rmse: 1.00161 + 0.00840504
[100]	cv_agg's rmse: 0.990962 + 0.00909816
[50]	cv_agg's rmse: 1.00155 + 0.00842064
[100]	cv_agg's rmse: 0.990961 + 0.00908677
[50]	cv_agg's rmse: 1.0014 + 0.00851521
[100]	cv_agg's rmse: 0.990836 + 0.0093396
[50]	cv_agg's rmse: 1.00173 + 0.00832245
[100]	cv_agg's rmse: 0.991121 + 0.00927051
[50]	cv_agg's rmse: 1.00173 + 0.0086257
[100]	cv_agg's rmse: 0.991155 + 0.00973057
[50]	cv_agg's rmse: 1.00155 + 0.0084091
[100]	cv_agg's rmse: 0.99055 + 0.00908532
[50]	cv_agg's rmse: 1.00149 + 0.00837795
[100]	cv_agg's rmse: 0.990758 + 0.00923502
[50]	cv_agg's rmse: 1.0015 + 0.00837332

[50]	cv_agg's rmse: 1.00149 + 0.00837795
[100]	cv_agg's rmse: 0.990758 + 0.00923502
[50]	cv_agg's rmse: 1.0015 + 0.00837332
[100]	cv_agg's rmse: 0.991013 + 0.00923856
[50]	cv_agg's rmse: 1.00144 + 0.00845832
[100]	cv_agg's rmse: 0.990716 + 0.00928783
[50]	cv_agg's rmse: 1.00149 + 0.00841725
[100]	cv_agg's rmse: 0.990835 + 0.00907928
[50]	cv_agg's rmse: 1.00161 + 0.00839547
[100]	cv_agg's rmse: 0.990934 + 0.00908839
[50]	cv_agg's rmse: 1.00161 + 0.00840504
[100]	cv_agg's rmse: 0.990962 + 0.00909816
[50]	cv_agg's rmse: 1.00155 + 0.00842064
[100]	cv_agg's rmse: 0.990961 + 0.00908677
[50]	cv_agg's rmse: 1.0014 + 0.00851521
[100]	cv_agg's rmse: 0.990836 + 0.0093396
[50]	cv_agg's rmse: 1.00173 + 0.00832245
[100]	cv_agg's rmse: 0.991121 + 0.00927051
[50]	cv_agg's rmse: 1.00173 + 0.0086257
[100]	cv_agg's rmse: 0.991155 + 0.00973057
[50]	cv_agg's rmse: 1.00155 + 0.0084091
[100]	cv_agg's rmse: 0.99055 + 0.00908532
[50]	cv_agg's rmse: 1.00149 + 0.00837795
[100]	cv_agg's rmse: 0.990758 + 0.0092350

[100]	cv_agg's rmse: 0.991447 + 0.00859528
[50]	cv_agg's rmse: 1.00393 + 0.00803131
[100]	cv_agg's rmse: 0.991526 + 0.00876869
[50]	cv_agg's rmse: 1.00457 + 0.00771814
[100]	cv_agg's rmse: 0.992126 + 0.00877516
[50]	cv_agg's rmse: 1.0039 + 0.00829602
[100]	cv_agg's rmse: 0.991529 + 0.00951331
[50]	cv_agg's rmse: 1.00395 + 0.00812785
[100]	cv_agg's rmse: 0.991902 + 0.00857748
[50]	cv_agg's rmse: 1.00383 + 0.00800058
[100]	cv_agg's rmse: 0.992051 + 0.00871607
[50]	cv_agg's rmse: 1.00391 + 0.00819607
[100]	cv_agg's rmse: 0.991778 + 0.0085265
[50]	cv_agg's rmse: 1.0042 + 0.00812042
[100]	cv_agg's rmse: 0.992629 + 0.0083592
[50]	cv_agg's rmse: 1.00358 + 0.00787795
[100]	cv_agg's rmse: 0.990887 + 0.00890264
[50]	cv_agg's rmse: 1.00359 + 0.00775086
[100]	cv_agg's rmse: 0.990875 + 0.00877017
[50]	cv_agg's rmse: 1.00396 + 0.0075568
[100]	cv_agg's rmse: 0.991208 + 0.00861533
[50]	cv_agg's rmse: 1.00393 + 0.00796044
[100]	cv_agg's rmse: 0.9915 + 0.00908137
[50]	cv_agg's rmse: 1.00447 + 0.00767491

[50]	cv_agg's rmse: 1.00256 + 0.00786036
[100]	cv_agg's rmse: 0.990765 + 0.00869918
[50]	cv_agg's rmse: 1.00256 + 0.00786036
[100]	cv_agg's rmse: 0.990765 + 0.00869918
[50]	cv_agg's rmse: 1.00256 + 0.00786036
[100]	cv_agg's rmse: 0.990765 + 0.00869918
[50]	cv_agg's rmse: 1.00256 + 0.00786036
[100]	cv_agg's rmse: 0.990765 + 0.00869918
[50]	cv_agg's rmse: 1.00256 + 0.00786036
[100]	cv_agg's rmse: 0.990765 + 0.00869918
[50]	cv_agg's rmse: 1.00256 + 0.00786036
[100]	cv_agg's rmse: 0.990765 + 0.00869918
[50]	cv_agg's rmse: 1.00256 + 0.00786036
[100]	cv_agg's rmse: 0.990765 + 0.00869918
[50]	cv_agg's rmse: 1.00256 + 0.00786036
[100]	cv_agg's rmse: 0.990765 + 0.00869918
[50]	cv_agg's rmse: 1.00256 + 0.00786036
[100]	cv_agg's rmse: 0.990765 + 0.00869918
[50]	cv_agg's rmse: 1.00229 + 0.00781045
[100]	cv_agg's rmse: 0.990996 + 0.0087148
[50]	cv_agg's rmse: 1.00205 + 0.00785451
[100]	cv_agg's rmse: 0.991337 + 0.00927826
[50]	cv_agg's rmse: 1.00265 + 0.00697242
[100]	cv_agg's rmse: 0.992096 + 0.00

[100]	cv_agg's rmse: 0.992407 + 0.00883888
[50]	cv_agg's rmse: 1.00181 + 0.00816038
[100]	cv_agg's rmse: 0.99099 + 0.00890034
[50]	cv_agg's rmse: 1.00192 + 0.00845783
[100]	cv_agg's rmse: 0.991013 + 0.00921056
[50]	cv_agg's rmse: 1.00211 + 0.0082656
[100]	cv_agg's rmse: 0.991049 + 0.00862194
[50]	cv_agg's rmse: 1.00224 + 0.00859082
[100]	cv_agg's rmse: 0.991141 + 0.00888799
[50]	cv_agg's rmse: 1.0026 + 0.0080601
[100]	cv_agg's rmse: 0.99198 + 0.00843175
[50]	cv_agg's rmse: 1.00218 + 0.00815705
[100]	cv_agg's rmse: 0.991184 + 0.00893541
[50]	cv_agg's rmse: 1.00226 + 0.00826614
[100]	cv_agg's rmse: 0.991115 + 0.00861806
[50]	cv_agg's rmse: 1.00172 + 0.00820472
[100]	cv_agg's rmse: 0.991343 + 0.00875228
[50]	cv_agg's rmse: 1.00207 + 0.00839575
[100]	cv_agg's rmse: 0.991721 + 0.00850535
[50]	cv_agg's rmse: 1.00212 + 0.00860241
[100]	cv_agg's rmse: 0.99181 + 0.00924808
[50]	cv_agg's rmse: 1.00181 + 0.00816038
[100]	cv_agg's rmse: 0.99099 + 0.00890034
[50]	cv_agg's rmse: 1.00175 + 0.0083907


[50]	cv_agg's rmse: 1.00283 + 0.00813683
[100]	cv_agg's rmse: 0.99358 + 0.00832721
[50]	cv_agg's rmse: 1.00321 + 0.00857405
[100]	cv_agg's rmse: 0.994095 + 0.00839289
[50]	cv_agg's rmse: 1.00155 + 0.0084091
[100]	cv_agg's rmse: 0.99055 + 0.00908532
[50]	cv_agg's rmse: 1.00156 + 0.00849592
[100]	cv_agg's rmse: 0.991515 + 0.00937015
[50]	cv_agg's rmse: 1.00208 + 0.00833279
[100]	cv_agg's rmse: 0.99267 + 0.00897278
[50]	cv_agg's rmse: 1.00184 + 0.0082363
[100]	cv_agg's rmse: 0.991853 + 0.00907911
[50]	cv_agg's rmse: 1.00214 + 0.00830323
[100]	cv_agg's rmse: 0.992509 + 0.00900168
[50]	cv_agg's rmse: 1.00182 + 0.00838779
[100]	cv_agg's rmse: 0.992402 + 0.00916217
[50]	cv_agg's rmse: 1.00219 + 0.00832529
[100]	cv_agg's rmse: 0.992602 + 0.00835098
[50]	cv_agg's rmse: 1.00243 + 0.00855357
[100]	cv_agg's rmse: 0.992457 + 0.00956872
[50]	cv_agg's rmse: 1.00203 + 0.00865945
[100]	cv_agg's rmse: 0.992284 + 0.00865239
[50]	cv_agg's rmse: 1.00266 + 0.00871706
[100]	cv_agg's rmse: 0.993317 + 0.008902

[100]	cv_agg's rmse: 0.990537 + 0.00918758
[50]	cv_agg's rmse: 1.00166 + 0.00840517
[100]	cv_agg's rmse: 0.990853 + 0.00907712
[50]	cv_agg's rmse: 1.00167 + 0.00836529
[100]	cv_agg's rmse: 0.990651 + 0.00917985
[50]	cv_agg's rmse: 1.00164 + 0.00835518
[100]	cv_agg's rmse: 0.99078 + 0.00909475
[50]	cv_agg's rmse: 1.00155 + 0.00844676
[100]	cv_agg's rmse: 0.990832 + 0.00915643
[50]	cv_agg's rmse: 1.00155 + 0.00844676
[100]	cv_agg's rmse: 0.99083 + 0.00915771
[50]	cv_agg's rmse: 1.00156 + 0.00828176
[100]	cv_agg's rmse: 0.990888 + 0.0091149
[50]	cv_agg's rmse: 1.00155 + 0.00844676
[100]	cv_agg's rmse: 0.990832 + 0.00915643
[50]	cv_agg's rmse: 1.00156 + 0.00828176
[100]	cv_agg's rmse: 0.990888 + 0.0091149
[50]	cv_agg's rmse: 1.00172 + 0.00830217
[100]	cv_agg's rmse: 0.990757 + 0.00912682
[50]	cv_agg's rmse: 1.00166 + 0.00837251
[100]	cv_agg's rmse: 0.990802 + 0.0092202
[50]	cv_agg's rmse: 1.00171 + 0.00841278
[100]	cv_agg's rmse: 0.990787 + 0.0093164
[50]	cv_agg's rmse: 1.00161 + 0.0083401

In [ ]:
# {'num_leaves': 95, 'max_depth': 5, 'min_data_in_leaf': 11, 'max_bin': 255, 'feature_fraction': 1.0, 'bagging_fraction': 1.0, 'bagging_freq': 45, 'lambda_l1': 0.1, 'lambda_l2': 0.001, 'min_split_gain': 1.0}

In [183]:
params = {

'boosting_type':'gbdt',
'objective':'regression',
'metric':'rmse',
'nthread':4,
'learning_rate':0.005,
"max_depth":5,
"num_leaves":95,
"max_bin":255,
"min_data_in_leaf":11,
"min_split_gain":1.0,
"feature_fraction": 1.0,
"bagging_fraction":1.0,
"bagging_freq":45,
"lambda_l1":0.1,
"lambda_l2":0.001,
}
train_data = lgb.Dataset(X_val, label=Y_val)
val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
lgb_r_cv = lgb.train(params, train_data, num_boost_round=8000, early_stopping_rounds=100, valid_sets=[train_data, val_data])

[1]	training's rmse: 1.80093	valid_1's rmse: 1.79264
Training until validation scores don't improve for 100 rounds
[2]	training's rmse: 1.79472	valid_1's rmse: 1.78644
[3]	training's rmse: 1.78855	valid_1's rmse: 1.78028
[4]	training's rmse: 1.78242	valid_1's rmse: 1.77416
[5]	training's rmse: 1.77633	valid_1's rmse: 1.76808
[6]	training's rmse: 1.77028	valid_1's rmse: 1.76205
[7]	training's rmse: 1.76427	valid_1's rmse: 1.75605
[8]	training's rmse: 1.7583	valid_1's rmse: 1.7501
[9]	training's rmse: 1.75237	valid_1's rmse: 1.74418
[10]	training's rmse: 1.74647	valid_1's rmse: 1.7383
[11]	training's rmse: 1.74062	valid_1's rmse: 1.73247
[12]	training's rmse: 1.73481	valid_1's rmse: 1.72667
[13]	training's rmse: 1.72903	valid_1's rmse: 1.72091
[14]	training's rmse: 1.7233	valid_1's rmse: 1.71519
[15]	training's rmse: 1.7176	valid_1's rmse: 1.70951
[16]	training's rmse: 1.71193	valid_1's rmse: 1.70387
[17]	training's rmse: 1.70631	valid_1's rmse: 1.69826
[18]	training's rmse: 1.70072	vali

[185]	training's rmse: 1.16006	valid_1's rmse: 1.15786
[186]	training's rmse: 1.15849	valid_1's rmse: 1.15632
[187]	training's rmse: 1.15693	valid_1's rmse: 1.15481
[188]	training's rmse: 1.15539	valid_1's rmse: 1.15332
[189]	training's rmse: 1.15386	valid_1's rmse: 1.15183
[190]	training's rmse: 1.15234	valid_1's rmse: 1.15036
[191]	training's rmse: 1.15084	valid_1's rmse: 1.14889
[192]	training's rmse: 1.14934	valid_1's rmse: 1.14745
[193]	training's rmse: 1.14786	valid_1's rmse: 1.14601
[194]	training's rmse: 1.1464	valid_1's rmse: 1.1446
[195]	training's rmse: 1.14494	valid_1's rmse: 1.14319
[196]	training's rmse: 1.1435	valid_1's rmse: 1.14179
[197]	training's rmse: 1.14207	valid_1's rmse: 1.1404
[198]	training's rmse: 1.14065	valid_1's rmse: 1.13903
[199]	training's rmse: 1.13924	valid_1's rmse: 1.13767
[200]	training's rmse: 1.13784	valid_1's rmse: 1.13631
[201]	training's rmse: 1.13646	valid_1's rmse: 1.13499
[202]	training's rmse: 1.13508	valid_1's rmse: 1.13366
[203]	training

[373]	training's rmse: 1.01269	valid_1's rmse: 1.01863
[374]	training's rmse: 1.01238	valid_1's rmse: 1.01836
[375]	training's rmse: 1.01207	valid_1's rmse: 1.01809
[376]	training's rmse: 1.01176	valid_1's rmse: 1.01782
[377]	training's rmse: 1.01145	valid_1's rmse: 1.01755
[378]	training's rmse: 1.01115	valid_1's rmse: 1.01728
[379]	training's rmse: 1.01085	valid_1's rmse: 1.01701
[380]	training's rmse: 1.01055	valid_1's rmse: 1.01676
[381]	training's rmse: 1.01026	valid_1's rmse: 1.0165
[382]	training's rmse: 1.00996	valid_1's rmse: 1.01624
[383]	training's rmse: 1.00967	valid_1's rmse: 1.01599
[384]	training's rmse: 1.00938	valid_1's rmse: 1.01574
[385]	training's rmse: 1.0091	valid_1's rmse: 1.01549
[386]	training's rmse: 1.00881	valid_1's rmse: 1.01525
[387]	training's rmse: 1.00853	valid_1's rmse: 1.015
[388]	training's rmse: 1.00825	valid_1's rmse: 1.01476
[389]	training's rmse: 1.00798	valid_1's rmse: 1.01452
[390]	training's rmse: 1.0077	valid_1's rmse: 1.01428
[391]	training'

[564]	training's rmse: 0.981696	valid_1's rmse: 0.993883
[565]	training's rmse: 0.98162	valid_1's rmse: 0.993836
[566]	training's rmse: 0.981545	valid_1's rmse: 0.993787
[567]	training's rmse: 0.981472	valid_1's rmse: 0.993736
[568]	training's rmse: 0.981401	valid_1's rmse: 0.993689
[569]	training's rmse: 0.981326	valid_1's rmse: 0.993646
[570]	training's rmse: 0.981252	valid_1's rmse: 0.9936
[571]	training's rmse: 0.981181	valid_1's rmse: 0.993552
[572]	training's rmse: 0.981112	valid_1's rmse: 0.993502
[573]	training's rmse: 0.981041	valid_1's rmse: 0.99346
[574]	training's rmse: 0.980971	valid_1's rmse: 0.993414
[575]	training's rmse: 0.9809	valid_1's rmse: 0.993372
[576]	training's rmse: 0.980831	valid_1's rmse: 0.993322
[577]	training's rmse: 0.980763	valid_1's rmse: 0.993276
[578]	training's rmse: 0.980691	valid_1's rmse: 0.993233
[579]	training's rmse: 0.980622	valid_1's rmse: 0.993193
[580]	training's rmse: 0.980555	valid_1's rmse: 0.993149
[581]	training's rmse: 0.980487	valid

[765]	training's rmse: 0.972129	valid_1's rmse: 0.988819
[766]	training's rmse: 0.972095	valid_1's rmse: 0.988803
[767]	training's rmse: 0.972061	valid_1's rmse: 0.988786
[768]	training's rmse: 0.972028	valid_1's rmse: 0.988773
[769]	training's rmse: 0.971999	valid_1's rmse: 0.98876
[770]	training's rmse: 0.971977	valid_1's rmse: 0.988748
[771]	training's rmse: 0.97194	valid_1's rmse: 0.988738
[772]	training's rmse: 0.971912	valid_1's rmse: 0.988731
[773]	training's rmse: 0.971879	valid_1's rmse: 0.988714
[774]	training's rmse: 0.971847	valid_1's rmse: 0.988703
[775]	training's rmse: 0.971822	valid_1's rmse: 0.988692
[776]	training's rmse: 0.971789	valid_1's rmse: 0.988674
[777]	training's rmse: 0.971758	valid_1's rmse: 0.988664
[778]	training's rmse: 0.971733	valid_1's rmse: 0.988654
[779]	training's rmse: 0.971712	valid_1's rmse: 0.988641
[780]	training's rmse: 0.971676	valid_1's rmse: 0.98863
[781]	training's rmse: 0.971644	valid_1's rmse: 0.988615
[782]	training's rmse: 0.97162	val

...

[1341]	training's rmse: 0.960798	valid_1's rmse: 0.985922
[1342]	training's rmse: 0.960787	valid_1's rmse: 0.985922
[1343]	training's rmse: 0.960777	valid_1's rmse: 0.985919
[1344]	training's rmse: 0.960763	valid_1's rmse: 0.985919
[1345]	training's rmse: 0.960747	valid_1's rmse: 0.985922
[1346]	training's rmse: 0.960737	valid_1's rmse: 0.985921
[1347]	training's rmse: 0.960719	valid_1's rmse: 0.98592
[1348]	training's rmse: 0.960712	valid_1's rmse: 0.985918
[1349]	training's rmse: 0.960698	valid_1's rmse: 0.985916
[1350]	training's rmse: 0.960686	valid_1's rmse: 0.985916
[1351]	training's rmse: 0.960676	valid_1's rmse: 0.985915
[1352]	training's rmse: 0.96067	valid_1's rmse: 0.985914
[1353]	training's rmse: 0.960652	valid_1's rmse: 0.985911
[1354]	training's rmse: 0.960643	valid_1's rmse: 0.98591
[1355]	training's rmse: 0.960633	valid_1's rmse: 0.98591
[1356]	training's rmse: 0.960609	valid_1's rmse: 0.985908
[1357]	training's rmse: 0.960604	valid_1's rmse: 0.985907
[1358]	training's 

[1528]	training's rmse: 0.958354	valid_1's rmse: 0.985562
[1529]	training's rmse: 0.958337	valid_1's rmse: 0.985558
[1530]	training's rmse: 0.958317	valid_1's rmse: 0.985552
[1531]	training's rmse: 0.95831	valid_1's rmse: 0.98555
[1532]	training's rmse: 0.958282	valid_1's rmse: 0.985546
[1533]	training's rmse: 0.958274	valid_1's rmse: 0.985545
[1534]	training's rmse: 0.958261	valid_1's rmse: 0.985548
[1535]	training's rmse: 0.958243	valid_1's rmse: 0.985542
[1536]	training's rmse: 0.958222	valid_1's rmse: 0.985539
[1537]	training's rmse: 0.958212	valid_1's rmse: 0.985539
[1538]	training's rmse: 0.958196	valid_1's rmse: 0.985537
[1539]	training's rmse: 0.958191	valid_1's rmse: 0.985539
[1540]	training's rmse: 0.958184	valid_1's rmse: 0.985539
[1541]	training's rmse: 0.958156	valid_1's rmse: 0.985534
[1542]	training's rmse: 0.958138	valid_1's rmse: 0.985529
[1543]	training's rmse: 0.958113	valid_1's rmse: 0.985526
[1544]	training's rmse: 0.958099	valid_1's rmse: 0.985524
[1545]	training'

[1713]	training's rmse: 0.95618	valid_1's rmse: 0.985236
[1714]	training's rmse: 0.95617	valid_1's rmse: 0.985236
[1715]	training's rmse: 0.956153	valid_1's rmse: 0.985232
[1716]	training's rmse: 0.956143	valid_1's rmse: 0.98523
[1717]	training's rmse: 0.956132	valid_1's rmse: 0.985222
[1718]	training's rmse: 0.956125	valid_1's rmse: 0.985222
[1719]	training's rmse: 0.956115	valid_1's rmse: 0.985221
[1720]	training's rmse: 0.956095	valid_1's rmse: 0.98522
[1721]	training's rmse: 0.956082	valid_1's rmse: 0.985219
[1722]	training's rmse: 0.956079	valid_1's rmse: 0.985218
[1723]	training's rmse: 0.956063	valid_1's rmse: 0.985217
[1724]	training's rmse: 0.956057	valid_1's rmse: 0.985217
[1725]	training's rmse: 0.956049	valid_1's rmse: 0.985215
[1726]	training's rmse: 0.956038	valid_1's rmse: 0.985214
[1727]	training's rmse: 0.956028	valid_1's rmse: 0.985207
[1728]	training's rmse: 0.956007	valid_1's rmse: 0.98521
[1729]	training's rmse: 0.955997	valid_1's rmse: 0.985211
[1730]	training's r

[1911]	training's rmse: 0.953876	valid_1's rmse: 0.985076
[1912]	training's rmse: 0.953872	valid_1's rmse: 0.985076
[1913]	training's rmse: 0.953861	valid_1's rmse: 0.985077
[1914]	training's rmse: 0.953842	valid_1's rmse: 0.985077
[1915]	training's rmse: 0.953833	valid_1's rmse: 0.985076
[1916]	training's rmse: 0.953824	valid_1's rmse: 0.98508
[1917]	training's rmse: 0.953814	valid_1's rmse: 0.985081
[1918]	training's rmse: 0.953807	valid_1's rmse: 0.98508
[1919]	training's rmse: 0.953789	valid_1's rmse: 0.985082
[1920]	training's rmse: 0.953773	valid_1's rmse: 0.985082
[1921]	training's rmse: 0.953759	valid_1's rmse: 0.98508
[1922]	training's rmse: 0.953749	valid_1's rmse: 0.985082
[1923]	training's rmse: 0.953731	valid_1's rmse: 0.985081
[1924]	training's rmse: 0.953712	valid_1's rmse: 0.985078
[1925]	training's rmse: 0.953699	valid_1's rmse: 0.985076
[1926]	training's rmse: 0.95369	valid_1's rmse: 0.985075
[1927]	training's rmse: 0.95368	valid_1's rmse: 0.985075
[1928]	training's r

[2053]	training's rmse: 0.953193	valid_1's rmse: 0.985032
[2054]	training's rmse: 0.953193	valid_1's rmse: 0.985032
[2055]	training's rmse: 0.953193	valid_1's rmse: 0.985032
[2056]	training's rmse: 0.953193	valid_1's rmse: 0.985032
[2057]	training's rmse: 0.953193	valid_1's rmse: 0.985032
[2058]	training's rmse: 0.953193	valid_1's rmse: 0.985032
[2059]	training's rmse: 0.953193	valid_1's rmse: 0.985032
[2060]	training's rmse: 0.953193	valid_1's rmse: 0.985032
[2061]	training's rmse: 0.953193	valid_1's rmse: 0.985032
[2062]	training's rmse: 0.953193	valid_1's rmse: 0.985032
[2063]	training's rmse: 0.953193	valid_1's rmse: 0.985032
[2064]	training's rmse: 0.953193	valid_1's rmse: 0.985032
[2065]	training's rmse: 0.953193	valid_1's rmse: 0.985032
[2066]	training's rmse: 0.953193	valid_1's rmse: 0.985032
Early stopping, best iteration is:
[1966]	training's rmse: 0.953193	valid_1's rmse: 0.985032


In [184]:
y_predict_lgb = lgb_r_cv.predict(x_test)
mse = mean_squared_error(np.expm1(y_predict_lgb), np.expm1(y_test))
mae = mean_absolute_error(np.expm1(y_predict_lgb), np.expm1(y_test))
rmse = mse ** 0.5
print(rmse)
print(mse)
print(mae)

3254.641073814273
10592688.519358926
666.548984086675


In [185]:
sum(np.expm1(y_predict_lgb))

11181658.924604056

In [186]:
sum(np.expm1(y_test))

18376283.0

In [187]:
sum(np.expm1(y_test))/sum(np.expm1(y_predict_lgb))

1.6434308293526048

## 用df_test数据集进行测试

In [188]:
df_test_pay = df_test[df_test['pay_sum']>0]
df_test_nopay = df_test[df_test['pay_sum']==0]
df_test_nopay_online_n = df_test_nopay[(df_test_nopay['role_created_online']<400) | (df_test_nopay['role_created_login_num']<3)|(df_test_nopay['max_role_level']<2)]
df_test_nopay_online_y = df_test_nopay.loc[np.setdiff1d(df_test_nopay.index, df_test_nopay_online_n.index),:]
df_test_nopay_online_y['pay_30'] = (df_test_nopay_online_y['role_created_30_pay_sum']>0).astype(int)
df_test_nopay_online_y.drop(['role_created_30_pay_sum'], axis=1, inplace=True)


In [189]:
df_test_part1 = df_test_nopay_online_n[['user_id','cp_server_no','cp_role_id','pay_num']].rename(columns=
                                                                                {'pay_num':'predict_30_pay'})

In [190]:

target_test = df_test_nopay_online_y['pay_30']
features_test = df_test_nopay_online_y.drop(['pay_30','user_id','cp_server_no','cp_role_id'], axis=1)
features_test = selector.fit_transform(features_test)  
y_predict = lgb_b.predict(features_test)

sessoin = pd.DataFrame(y_predict, columns=['pay_30'])
sessoin['pay_30']=sessoin['pay_30'].apply(lambda x:1 if x>=0.4 else 0)
y_predict_val = sessoin['pay_30'].values
print(recall_score(target_test, y_predict_val, average=None))
print(precision_score(target_test, y_predict_val))
print(accuracy_score(target_test, y_predict_val))
print(f1_score(target_test, y_predict_val))

[0.83967269 0.78494313]
0.11285583612828118
0.8382866321961985
0.19733908491663435


In [191]:
sessoin = pd.DataFrame(y_predict, columns=['pay_30'])
sessoin['pay_30']=sessoin['pay_30'].apply(lambda x:1 if x>=0.6 else 0)
y_predict_val = sessoin['pay_30'].values
print(recall_score(target_test, y_predict_val, average=None))
print(precision_score(target_test, y_predict_val))
print(accuracy_score(target_test, y_predict_val))
print(f1_score(target_test, y_predict_val))

[0.90415076 0.57904376]
0.1356747600225861
0.89591725134635
0.21983933832275065


In [192]:
target_test[target_test>0].shape

(10374,)

In [193]:
y_predict[y_predict>0.8].shape

(12962,)

In [194]:
df_test_part2 = df_test_nopay_online_y[['user_id','cp_server_no','cp_role_id']]
df_test_part2['predict_30_pay'] = y_predict
df_test_part2['predict_30_pay'] = df_test_part2['predict_30_pay'].apply(lambda x:250 if x>0.8 else 0)

In [195]:
target_test = df_test_pay['role_created_30_pay_sum']
target_test_ln = np.log1p(target_test)
features_test = df_test_pay.drop(['role_created_30_pay_sum','user_id','cp_server_no','cp_role_id'], axis=1)
y_predict = lgb_r_cv.predict(features_test)
y_predict[y_predict<0] = 0
mse = mean_squared_error(np.expm1(y_predict), np.expm1(target_test_ln))
mae = mean_absolute_error(np.expm1(y_predict), np.expm1(target_test_ln))
rmse = mse ** 0.5
print(rmse)
print(mse)
print(mae)

3299.6981822115185
10888008.093689999
690.0462435633906


In [196]:
sum(np.expm1(y_predict))

16169957.456412932

In [197]:
sum(np.expm1(target_test_ln))

27090266.0

In [198]:
df_test_part3 = df_test_pay[['user_id','cp_server_no','cp_role_id']]
df_test_part3['predict_30_pay'] = np.expm1(y_predict) *1.76

In [200]:
pred = df_test_part1.append(df_test_part2)
pred = pred.append(df_test_part3)
pred['user_id'] = pred['user_id'].astype(str)
pred['cp_server_no'] = pred['cp_server_no'].astype(str)
pred['cp_role_id'] = pred['cp_role_id'].astype(str)
df_test['user_id'] = df_test['user_id'].astype(str)
df_test['cp_server_no'] = df_test['cp_server_no'].astype(str)
df_test['cp_role_id'] = df_test['cp_role_id'].astype(str)

In [205]:
predict_data = pd.merge(df_test[['user_id','cp_server_no','cp_role_id', 'role_created_30_pay_sum']],pred,on=['user_id','cp_server_no','cp_role_id'],how='left',validate='one_to_one')

In [208]:
mse = mean_squared_error(predict_data['predict_30_pay'], predict_data['role_created_30_pay_sum'])
mae = mean_absolute_error(predict_data['predict_30_pay'], predict_data['role_created_30_pay_sum'])
rmse = mse ** 0.5
print('测试集上的均方根误差:%.2f元'% rmse)
# print('测试集上的均方误差:%f元' % mse)
print('测试集上的平均绝对误差:%.2f元'% mae)

测试集上的均方根误差:448.68元
测试集上的平均绝对误差:18.43元


In [209]:
print('测试集前30天实际总的付费金额:%.2f元' % predict_data['role_created_30_pay_sum'].sum())
print('测试集前30天预测总的付费金额:%.2f元'% predict_data['predict_30_pay'].sum())
print('预测总金额准确率:',predict_data['predict_30_pay'].sum()/predict_data['role_created_30_pay_sum'].sum())

测试集前30天实际总的付费金额:29795062.00元
测试集前30天预测总的付费金额:31699625.12元
预测总金额准确率: 1.063922106397591


# 分计划

In [210]:
data['create_role_time'] = data['create_role_time'].dt.date
df_source = data[['user_id','cp_server_no','cp_role_id','create_role_time','channel_id','source_id','pay_sum']]
df_source.drop_duplicates(inplace=True)
df_source_predict = pd.merge(predict_data,df_source,on=['user_id','cp_server_no','cp_role_id'],how='left',validate='one_to_one')

In [211]:
df_source_predict['is_pay'] = (df_source_predict['pay_sum'] != 0).astype(int)
groups = df_source_predict.groupby(['channel_id','source_id','create_role_time'])
print(groups.ngroups)
temp1 = groups['role_created_30_pay_sum'].agg([('30_pay_sum','sum')]).reset_index()
temp2 = groups['predict_30_pay'].agg([('predict_30_pay','sum')]).reset_index()
temp3 = groups['pay_sum'].agg([('n_pay_sum','sum')]).reset_index()
temp4 = groups['user_id'].agg([('n_user_sum','count')]).reset_index()
temp5 = groups['is_pay'].agg([('pay_user_sum','sum')]).reset_index()

125794


In [212]:
df_source_predict = pd.merge(temp1,temp2,on=['channel_id','source_id','create_role_time'],how='inner',validate='one_to_one')
df_source_predict = pd.merge(df_source_predict,temp3,on=['channel_id','source_id','create_role_time'],how='inner',validate='one_to_one')
df_source_predict = pd.merge(df_source_predict,temp4,on=['channel_id','source_id','create_role_time'],how='inner',validate='one_to_one')
df_source_predict = pd.merge(df_source_predict,temp5,on=['channel_id','source_id','create_role_time'],how='inner',validate='one_to_one')

In [213]:
def weight_error(df):
    df['error'] = np.abs((df['predict_30_pay']-df['30_pay_sum'])/df['30_pay_sum'])
    df['weight'] = df['30_pay_sum'] / df['30_pay_sum'].sum()
    df['weight_error'] = df_source_predict['weight'] * df_source_predict['error']
    return df['weight_error'].sum()

In [214]:
weight_error(df_source_predict)

0.7543702436609601

In [215]:
print(weight_error(df_source_predict[df_source_predict['n_user_sum']>50]))
print(df_source_predict[df_source_predict['n_user_sum']>4].shape[0])

0.394852351607182
32774


In [216]:
print(weight_error(df_source_predict[df_source_predict['pay_user_sum']>5]))
print(df_source_predict[df_source_predict['pay_user_sum']>5].shape[0])

0.14092917640750888
712


In [217]:
print(weight_error(df_source_predict[df_source_predict['pay_user_sum']>4]))
print(df_source_predict[df_source_predict['pay_user_sum']>4].shape[0])

0.1760248063913997
987


In [218]:
print(weight_error(df_source_predict[df_source_predict['pay_user_sum']>3]))
print(df_source_predict[df_source_predict['pay_user_sum']>3].shape[0])

0.23268805605318818
1440


In [219]:
print(weight_error(df_source_predict[df_source_predict['pay_user_sum']>2]))
print(df_source_predict[df_source_predict['pay_user_sum']>2].shape[0])

0.30579253772620363
2343
